## Preparation

In [1]:
from subprocess import getoutput as gop

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors

In [3]:
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw

In [4]:
def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None
def NeutraliseCharges(smiles, reactions=None):
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
    mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            mol = rms[0]
    if replaced:
        return Chem.MolToSmiles(mol,True)
    else:
        return smiles

In [5]:
def issubstructure(m1, m2):
    res = rdFMCS.FindMCS([m1, m2])
    return m1.GetNumAtoms() == res.numAtoms or m2.GetNumAtoms() == res.numAtoms

## Read fragments.csv

In [6]:
fragments = pd.read_csv("fragments.csv", names=["SMILES", "cnt"])
fragments.head()

,SMILES,cnt
0,OC=O,1
1,CN[C@H]1[C@H](OC)C=CC[C@@H]1N,1
2,CCC,1
3,CC=O,1
4,CC,1


## Unite some records which canonical smiles are same

In [7]:
fragments["CANONICAL_SMILES"] = fragments.SMILES.apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
for idx, item in fragments.iterrows():
    smiles_neut = NeutraliseCharges(item.CANONICAL_SMILES)
    if (fragments.CANONICAL_SMILES == smiles_neut).any():
        fragments.loc[idx, "CANONICAL_SMILES"] = smiles_neut

In [8]:
fragments = fragments.groupby("CANONICAL_SMILES").sum().sort_values("cnt", ascending=False).reset_index()

## Calculate several descriptors

In [9]:
fragments["mol"] = fragments.CANONICAL_SMILES.apply(Chem.MolFromSmiles)
fragments["NumCuttingSections"] = fragments.mol.apply(lambda m: sum([a.GetIsotope() == 1 for a in m.GetAtoms()]))
fragments["NumAliphaticRings"] = fragments.mol.apply(rdMolDescriptors.CalcNumAliphaticRings)
fragments["NumAromaticRings"] = fragments.mol.apply(rdMolDescriptors.CalcNumAromaticRings)
fragments["NumLargestRingAtoms"] = fragments.mol.apply(lambda m: ([len(l) for l in m.GetRingInfo().AtomRings()]))
fragments["NumLargestRingAtoms"] = fragments.NumLargestRingAtoms.apply(lambda l: max(l) if len(l) != 0 else 0)
fragments["NumRings"] = fragments.NumAliphaticRings + fragments.NumAromaticRings
fragments["NumAtoms"] = fragments.mol.apply(lambda m: m.GetNumAtoms()) - fragments.NumCuttingSections
fragments["NumHeteroAtoms"] = fragments.mol.apply(rdMolDescriptors.CalcNumHeteroatoms)
fragments["NumHBA"] = fragments.mol.apply(rdMolDescriptors.CalcNumHBA)
fragments["NumHBD"] = fragments.mol.apply(rdMolDescriptors.CalcNumHBD)
fragments["NumDoubleBonds"] = fragments.CANONICAL_SMILES.apply(lambda s: s.count("="))
fragments["NumTripleBonds"] = fragments.CANONICAL_SMILES.apply(lambda s: s.count("#"))
fragments["ExistBoron"] = fragments.mol.apply(lambda m: sum([(a.GetAtomicNum() == 5) for a in m.GetAtoms()]) != 0)
fragments["ExistSilicon"] = fragments.mol.apply(lambda m: sum([(a.GetAtomicNum() == 14) for a in m.GetAtoms()]) != 0)

## Substructure filtering

- Filtering based on properties of each molecule
- Filtering based on substructural relationship

In [10]:
fragments = fragments.query("NumAliphaticRings <= 1 and NumAromaticRings <= 2")
print(len(fragments))
fragments = fragments.query("NumAliphaticRings == 0 or NumAromaticRings == 0")
print(len(fragments))
fragments = fragments.query("NumRings <= 2")
print(len(fragments))
fragments = fragments.query("not (NumRings == 0 and NumAtoms > 6)")
print(len(fragments))
fragments = fragments.query("not (NumRings == 0 and NumHeteroAtoms == 0)")
print(len(fragments))
fragments = fragments.query("NumLargestRingAtoms <= 6")
print(len(fragments))
fragments = fragments.query("cnt >= 5")
print(len(fragments))
fragments = fragments.query("ExistBoron == False and ExistSilicon == False")
print(len(fragments))


5
5
5
5
3
3
0
0


In [11]:
fragments["deleteflag"] = False
for idx, item in fragments[fragments.NumAtoms < 3].iterrows():
    temp = fragments[fragments.mol.apply(lambda m: issubstructure(m, item.mol))]
    temp = temp.query("NumAtoms == %d + 1 and NumHeteroAtoms == %d and NumHBA == %d and NumHBD == %d and NumCuttingSections == %d" % (item.NumAtoms, item.NumHeteroAtoms, item.NumHBA, item.NumHBD, item.NumCuttingSections))
    temp = temp.query("NumDoubleBonds == %d and NumTripleBonds == %d" % (item.NumDoubleBonds, item.NumTripleBonds))
    # temp = temp.query("NumAliphaticRings == %d and NumAromaticRings == %d" % (item.NumAliphaticRings, item.NumAromaticRings)) 
    # The above filtering is not needed because the condition is always true for fragments with fragments.NumAtoms < 3 
    if(len(temp) != 0):
        fragments.loc[idx, "deleteflag"] = True
fragments = fragments[fragments.deleteflag == False]
del fragments["deleteflag"]

print(len(fragments))

0


In [12]:
fragments["deleteflag"] = False
for idx, item in fragments.query("NumAtoms > 5 and NumRings == 0").iterrows():
    temp = fragments[fragments.mol.apply(lambda m: issubstructure(m, item.mol))]
    temp = temp.query("NumAtoms == %d + 1 and NumHeteroAtoms == %d and NumHBA == %d and NumHBD == %d and NumCuttingSections == %d" % (item.NumAtoms, item.NumHeteroAtoms, item.NumHBA, item.NumHBD, item.NumCuttingSections))
    temp = temp.query("NumDoubleBonds == %d and NumTripleBonds == %d" % (item.NumDoubleBonds, item.NumTripleBonds))
    temp = temp.query("NumAliphaticRings == %d and NumAromaticRings == %d" % (item.NumAliphaticRings, item.NumAromaticRings))
    if(len(temp) != 0):
        fragments.deleteflag[idx] = True
fragments = fragments[fragments.deleteflag == False]
del fragments["deleteflag"]

print(len(fragments))

0


## Visualization and Output of selected substructures

- substructure_list_000~999.pdf
- cosolvent_substructures.smi

In [13]:
fragments = fragments.sort_values(["cnt", "CANONICAL_SMILES"], ascending=False)
imgs=[]
mols = list(fragments.mol)
struct_per_page = 90
for i in range(0, len(mols), struct_per_page):
    imgs.append(Draw.MolsToGridImage(mols[i:i+struct_per_page], 
                                     molsPerRow=6, subImgSize=(300,200), 
                                     legends=[f"counts: {n}" for n in fragments.cnt][i:i+struct_per_page]))

for i in range(len(imgs)):
    imgs[i].save("substructure_list_%03d.pdf"%i)

In [14]:
fragments[["CANONICAL_SMILES"]].to_csv("cosolvent_substructures.smi", header=None, index=None)